In [ ]:
import random

print(random.random())

0.013436375940380607


In [ ]:
import pandas as pd

train= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/국립국어원/SKT_BoolQ_Train.tsv", delimiter= '\t')

train.tail(10)

,ID,Text,Question,"Answer(FALSE = 0, TRUE = 1)"
3655,3656,방과 후 친구의 부탁으로 청소당번을 하고 있으려니 전날의 고양이 신하 '나토루'가 ...,하루는 고양이 왕의 왕비가 되어달라는 제안을 받았다.,0
3656,3657,하루가 다시 인간이 되어 인간 세계로 가려면 동이 트기 전 미로를 지나고 탑을 올라...,룬 왕자는 하루가 아닌 유키를 사랑하고 있었다.,1
3657,3658,고양이 왕은 상황이 이렇게 됐으니 자기의 아내라고 되라고 하지만 하루는 단칼에 거절...,탑이 무너지자 하루 일행은 탈출에 실패했나요?,0
3658,3659,"대학수학능력시험(大學修學能力試驗, 영어: College Scholastic Abil...",대학수학능력시험은 1994년부터 실시되었는가?,1
3659,3660,수능 문제를 내는 동안 출제진이 숙소에서 합숙한다는 사실을 외부에서 알지 못하도록 ...,수능 출제에 대한 모든 내용은 국가기밀인가?,1
3660,3661,"사법시험은 법조인이 될 자격을 검정하는 시험이다. 제1차(객관식), 제2차(서술형 ...",사법 시험 합격 이후에는 사법연수원을 들어가야 하는가?,1
3661,3662,한국사능력검정시험(韓國史能力檢定試驗)은 한국사에 대한 관심을 확산 · 심화시키는 계...,한국사능력검정시험(韓國史能力檢定試驗)은 국사편찬위원회가 주관하는가?,1
3662,3663,"한국교통안전공단(韓國交通安全公團, Korea Transportation Safety...",한국교통안전공단은 준정부기관인가?,1
3663,3664,"변호사시험은 사법시험의 후신으로서, 조선변호사시험-고등고시 사법과-사법시험-변호사시...",변호사 시험은 2017년에 완전히 폐지되었는가?,1
3664,3665,OPIc(Oral Proficiency Interview-Computer)는 공인인...,OPIc(Oral Proficiency Interview-Computer)은 공인인...,1


In [ ]:
train.Text[9]

'3차원 프린터는 밀링 또는 절삭이 아닌, 기존 잉크젯 프린터에서 쓰이는 것과 유사한 적층 방식으로 입체물로 제작하는 장치를 말하며, 컴퓨터로 제어되기 때문에 만들 수 있는 형태가 다양하고 다른 제조 기술에 비해 사용하기 쉽다. 단점으로는 현재 기술로는 제작 속도가 매우 느리다는 점과, 적층 구조로 인해 표면이 매끄럽지 못하다는 점 그리고 위험한 총기와 같은 물건을 마음대로 인쇄할 수 있다는 점, 지식재산권을 침해할 수도 있다는 점 등이 있다.'

In [1]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 25.8 MB/s 
     |████████████████████████████████| 3.3 MB 38.4 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 39.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [ ]:
from google.colab import files 
src = list(files.upload().values())[0] 
open('dataset.py','wb').write(src) 



Saving dataset.py to dataset (1).py


1568

In [ ]:
#####개체명 인식

import numpy
import torch
from torch.utils.data import Dataset

class SKT_BoolQ(Dataset):
  def __init__(self, fileName):
    self.data = []
    readDatas = open(fileName,'r').read().strip().split("\n")

    for readData in readDatas[1:]:
      data_format = len(readData.split("\t"))

      if data_format == 6:
        data_i = readData.split("\t")
        if data_i[3] == '0':

          #text ner의 경우 data_i[5]
          #question ner    data_i[4]
          self.data.append(["skt_boolq question: " + data_i[2] + "NER: " +data_i[4] + "text: " + data_i[1], "false"])
        else:
          self.data.append(["skt_boolq question: " + data_i[2] + "NER: " +data_i[4] + "text: " + data_i[1], "true"])

      else:
        print("Incorrectly formatted.")

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    text = self.data[idx][0]
    label = self.data[idx][1]

    return {'text' : text, 'label' : label}


class T5_Classification_Collator(object):
  def __init__(self, use_tokenizer, max_sequence_len=None):
    self.use_tokenizer = use_tokenizer
    self.max_sequence_len = use_tokenizer.model_max_length if max_sequence_len is None else max_sequence_len

  def __call__(self, sequences):
    texts = [sequence['text'] for sequence in sequences]
    labels = [sequence['label'] for sequence in sequences]
    inputs = self.use_tokenizer(text=texts, return_tensors="pt", padding=True, truncation=True,  max_length=self.max_sequence_len)
    labels = self.use_tokenizer(text=labels, return_tensors="pt", padding=True, truncation=True,  max_length=self.max_sequence_len).input_ids
    inputs.update({'labels':torch.tensor(labels)})

    return inputs

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
#from dataset import SKT_BoolQ, T5_Classification_Collator
from torch.utils.data import DataLoader
from transformers import AdamW, get_constant_schedule_with_warmup
import torch
import random
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score

random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print("Load T5 model...")
tokenizer = T5Tokenizer.from_pretrained("KETI-AIR/ke-t5-base")
model = T5ForConditionalGeneration.from_pretrained("KETI-AIR/ke-t5-base")
model.cuda()



In [ ]:
batch_size = 8

collator = T5_Classification_Collator(use_tokenizer=tokenizer, max_sequence_len=512)

print("Load Dataset...")
train_dataset = SKT_BoolQ('/content/drive/MyDrive/Colab Notebooks/국립국어원/SKT_BoolQ_NER_Train.tsv')
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collator)

dev_dataset = SKT_BoolQ('/content/drive/MyDrive/Colab Notebooks/국립국어원/SKT_BoolQ_NER_Dev.tsv')
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, collate_fn=collator)

epochs = 10
total_steps = len(train_dataloader) * epochs

optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )
#scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                            num_warmup_steps = len(train_dataloader), # Default value in run_glue.py
#                                            num_training_steps = total_steps)
scheduler = get_constant_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = len(train_dataloader))

print("Training...")

for epoch_i in range(epochs):
  print("-------------------------------------------------------------")
  # train
  model.train()

  true_labels = []
  prediction_labels = []
  total_loss = 0

  for batch in tqdm(train_dataloader):
    true_labels += batch['labels'].numpy().flatten().tolist()
    batch = {k:v.type(torch.long).cuda() for k,v in batch.items()}

    model.zero_grad()
    outputs = model(**batch)

    loss, logits = outputs[:2]
    total_loss += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()

    logits = logits.detach().cpu().numpy()

    prediction_labels += logits.argmax(axis=-1).flatten().tolist()

  train_avg_epoch_loss = total_loss / len(train_dataloader)
  train_acc = accuracy_score(true_labels, prediction_labels)

  prediction_labels = []
  true_labels = []
  total_loss = 0

  model.eval()

  # Evaluate data for one epoch
  for batch in tqdm(dev_dataloader):
    true_labels += (batch['labels'].numpy())[:, 0].flatten().tolist()
    batch = {k:v.type(torch.long).cuda() for k,v in batch.items()}

    with torch.no_grad():        
        outputs = model(**batch)

        loss, logits = outputs[:2]
        
        logits = logits.detach().cpu().numpy()
        total_loss += loss.item()

        predict_content = (logits.argmax(axis=-1))[:, 0].flatten().tolist()
        prediction_labels += predict_content

  # Calculate the average loss over the training data.
  dev_avg_epoch_loss = total_loss / len(dev_dataloader)
  #print(len(true_labels))
  #print(len(prediction_labels))
  #print(len(dev_dataloader))
  dev_acc = accuracy_score(true_labels, prediction_labels)

  print("Epoch: %d  train_loss: %.5f  train_acc: %.5f  dev_loss: %.5f  dev_acc: %.5f"%(epoch_i, train_avg_epoch_loss, train_acc, dev_avg_epoch_loss, dev_acc))


Load Dataset...
Training...
-------------------------------------------------------------


  0%|          | 0/459 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  2%|▏         | 11/459 [00:03<02:13,  3.36it/s]


KeyboardInterrupt: ignored

In [ ]:
!nvidia-smi

Wed Oct  6 04:20:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd 'drive/MyDrive/Colab Notebooks/국립국어원/'

/content/drive/MyDrive/Colab Notebooks/국립국어원


In [ ]:
!python train_SKT_BoolQ.py

Load T5 model...
Traceback (most recent call last):
  File "train_SKT_BoolQ.py", line 24, in <module>
    model.cuda()
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 637, in cuda
    return self._apply(lambda t: t.cuda(device))
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 530, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 552, in _apply
    param_applied = fn(param)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 637, in <lambda>
    return self._apply(lambda t: t.cuda(device))
RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.


In [ ]:
###########모델 저장
torch.save(model.state_dict(), 'drive/MyDrive/Colab Notebooks/국립국어원/'+ 'model_large.pt')

In [ ]:
import pandas as pd

test= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/국립국어원/SKT_BoolQ_Test.tsv", delimiter= '\t')

test.tail()

,ID,Text,Question,"Answer(FALSE = 0, TRUE = 1)"
699,700,이전에 치러지던 학력고사는 고등학교 과정의 많은 과목별로 문제가 출제되었기 때문에 ...,학력고사는 교과서를 암기해야 하는 방식의 시험이었나요?,NaN
700,701,"대학수학능력시험을 응시하는 수험생 중 시각장애인, 청각장애인, 뇌병변장애인 등 장애...",시각장애인은 대학수학능력시험을 응시할 수 없는가?,NaN
701,702,"한국어능력시험(韓國語能力試驗, 영어: Test of Proficiency in Ko...",한국어능력시험은 한국어를 모국어로 하는 사람을 대상으로 하는가?,NaN
702,703,KBS 한국어능력시험은 한국방송공사가 주관하는 한국어 능력 시험이다. 문화체육관광부...,KBS 한국어능력시험 결과는 국가공인 자격증 시험인가요?,NaN
703,704,"국가영어능력평가시험(NEAT, National English Ability Test...",국가영어능력평가시험 3급은 국립국제교육원에서 주관하는가?,NaN


In [ ]:
###########모델 base 불러오기
device = torch.device('cuda')
model = T5ForConditionalGeneration.from_pretrained("KETI-AIR/ke-t5-base")
model.load_state_dict(torch.load('drive/MyDrive/Colab Notebooks/국립국어원/model.pt'))
model.to(device)
model.eval()

In [ ]:
#######################dataset 모듈
from torch.utils.data import Dataset

class SKT_BoolQ(Dataset):
  def __init__(self, fileName):
    self.data = []
    readDatas = open(fileName,'r').read().strip().split("\n")

    for readData in readDatas[1:]:
      data_format = len(readData.split("\t"))
      
      data_i = readData.split("\t")

      self.data.append(["skt_boolq question: " + data_i[2] + "text: " + data_i[1], "nan"])

    print(len(self.data))


  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    text = self.data[idx][0]
    label = self.data[idx][1]

    return {'text' : text, 'label' : label}


class T5_Classification_Collator(object):
  def __init__(self, use_tokenizer, max_sequence_len=None):
    self.use_tokenizer = use_tokenizer
    self.max_sequence_len = use_tokenizer.model_max_length if max_sequence_len is None else max_sequence_len

  def __call__(self, sequences):
    texts = [sequence['text'] for sequence in sequences]
    labels = [sequence['label'] for sequence in sequences]
    inputs = self.use_tokenizer(text=texts, return_tensors="pt", padding=True, truncation=True,  max_length=self.max_sequence_len)
    labels = self.use_tokenizer(text=labels, return_tensors="pt", padding=True, truncation=True,  max_length=self.max_sequence_len).input_ids
    inputs.update({'labels':torch.tensor(labels)})

    return inputs


In [ ]:
###############모델 base 평가

batch_size = 1

collator = T5_Classification_Collator(use_tokenizer=tokenizer, max_sequence_len=512)

print("Load Dataset...")
test_dataset = SKT_BoolQ('/content/drive/MyDrive/Colab Notebooks/국립국어원/SKT_BoolQ_Test.tsv')
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collator)

epochs = 1
total_steps = len(test_dataloader) * epochs

optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )
#scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                            num_warmup_steps = len(test_dataloader), # Default value in run_glue.py
#                                            num_training_steps = total_steps)
scheduler = get_constant_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = len(test_dataloader))

print("Testing...")

true_labels = []
prediction_labels = []
total_loss = 0

for epoch_i in range(1):
  print("-------------------------------------------------------------")
  # test
  model.eval()

  # Evaluate data for one epoch
  for batch in tqdm(test_dataloader):
    #true_labels += (batch['labels'].numpy())[:, 0].flatten().tolist()
    batch = {k:v.type(torch.long).cuda() for k,v in batch.items()}

    with torch.no_grad():        
        outputs = model(**batch)

        loss, logits = outputs[:2]
        
        logits = logits.detach().cpu().numpy()
        total_loss += loss.item()

        predict_content = (logits.argmax(axis=-1))[:, 0].flatten().tolist()
        prediction_labels += predict_content

  # Calculate the average loss over the training data.
  test_avg_epoch_loss = total_loss / len(test_dataloader)
  #print((true_labels))
  #print((prediction_labels))
  #print(len(test_dataloader))
  #test_acc = accuracy_score(true_labels, prediction_labels)

  print("Epoch: %d  test_loss: %.5f"%(epoch_i, test_avg_epoch_loss))


Load Dataset...
704
Testing...
-------------------------------------------------------------


100%|██████████| 704/704 [00:16<00:00, 43.67it/s]

Epoch: 0  test_loss: 22.79664


In [ ]:
logits.shape

(1, 3, 64128)

In [ ]:
tokenizer._convert_id_to_token(12958)

'▁false'

In [ ]:
test['Answer(FALSE = 0, TRUE = 1)']=prediction_labels

test['Answer(FALSE = 0, TRUE = 1)'] = test['Answer(FALSE = 0, TRUE = 1)'].apply(lambda x: 1 if x==3966 else x)
test['Answer(FALSE = 0, TRUE = 1)'] = test['Answer(FALSE = 0, TRUE = 1)'].apply(lambda x: 0 if x==12958 else x)

test.head(10)

,ID,Text,Question,"Answer(FALSE = 0, TRUE = 1)"
0,1,콜라독립 815(815 Cola)는 코카콜라를 대한민국에서 라이선스로 만들어 오던 ...,콜라독립 815는 코카콜라에서 만들었나요?,0
1,2,주택임대차보호법은 주거용 건물의 임대차에 관하여 민법에 대한 특례를 규정함으로써 국...,토지는 주택임대차보호법에 적용되나요?,0
2,3,"비례대표제는 정당의 득표율에 비례해 당선자 수를 결정하는 선거 제도로, 각 정당을 ...",비례대표제는 정당의 의석률과 정당 득표율이 거의 비례하나요?,1
3,4,사회적 거리두기는 기침이나 재채기 등의 비말 감염의 경우에 가장 효과적이다. 성적 ...,사회적 거리두기는 공기감염에는 효과가 없나요?,0
4,5,고위공직자범죄수사처는 고위 공직자 및 대통령 친인척의 범죄행위를 상시적으로 수사·기...,고위공직자범죄수사처는 대통령 친인척을 수사할 수 있나?,1
5,6,보통선거는 연령 이외의 자격 조건을 두지 않고 국민 모두에게 선거권이 주어지는 선거...,범죄자도 보통선거의 투표권이 있는가?,0
6,7,재산세 또는 주택세는 모두 부속토지에 귀속된 건물에 대한 지방세이다. 이는 소유자에...,주택세와 재산세는 같나?,0
7,8,"신간회는 1927년 2월 15일에 사회주의, 민족주의 세력들이 결집해서 창립한 항일...",신간회 사회주의계열 독립운동단체인가요?,1
8,9,대법원은 많은 사법 관할 구역에서 법정의 계층 내에서 가장 높은 법정을 말한다. 대...,대법원이 우리나라 최고 법원인가요?,0
9,10,"찰스 에번스 휴즈는 미국의 정치인이고, 공화당 정치인이며, 미국의 제11대 대법원장...",미국 대법원장이 대통령 선거에 출마할 수 있는가?,1


In [ ]:
test.to_excel('/content/drive/MyDrive/Colab Notebooks/test.xlsx', header= False)

In [ ]:
#######################dev set
#######################dataset
import numpy
import torch
from torch.utils.data import Dataset

class SKT_BoolQ(Dataset):
  def __init__(self, fileName):
    self.data = []
    readDatas = open(fileName,'r').read().strip().split("\n")

    for readData in readDatas[1:]:
      data_format = len(readData.split("\t"))

      if data_format == 4:
        data_i = readData.split("\t")
        if data_i[3] == '0':
          self.data.append(["skt_boolq question: " + data_i[2] + "text: " + data_i[1], "false"])
        else:
          self.data.append(["skt_boolq question: " + data_i[2] + "text: " + data_i[1], "true"])

      else:
        print("Incorrectly formatted.")

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    text = self.data[idx][0]
    label = self.data[idx][1]

    return {'text' : text, 'label' : label}


class T5_Classification_Collator(object):
  def __init__(self, use_tokenizer, max_sequence_len=None):
    self.use_tokenizer = use_tokenizer
    self.max_sequence_len = use_tokenizer.model_max_length if max_sequence_len is None else max_sequence_len

  def __call__(self, sequences):
    texts = [sequence['text'] for sequence in sequences]
    labels = [sequence['label'] for sequence in sequences]
    inputs = self.use_tokenizer(text=texts, return_tensors="pt", padding=True, truncation=True,  max_length=self.max_sequence_len)
    labels = self.use_tokenizer(text=labels, return_tensors="pt", padding=True, truncation=True,  max_length=self.max_sequence_len).input_ids
    inputs.update({'labels':torch.tensor(labels)})

    return inputs

In [ ]:
###############모델 base validation 평가

batch_size = 1

collator = T5_Classification_Collator(use_tokenizer=tokenizer, max_sequence_len=512)

print("Load Dataset...")
dev_dataset = SKT_BoolQ('/content/drive/MyDrive/Colab Notebooks/국립국어원/SKT_BoolQ_Dev.tsv')
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, collate_fn=collator)

epochs = 1
total_steps = len(dev_dataloader) * epochs

optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )
#scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                            num_warmup_steps = len(test_dataloader), # Default value in run_glue.py
#                                            num_training_steps = total_steps)
scheduler = get_constant_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = len(dev_dataloader))

print("Testing...")

true_labels = []
prediction_labels = []
total_loss = 0

for epoch_i in range(1):
  print("-------------------------------------------------------------")
  # test
  model.eval()

  # Evaluate data for one epoch
  for batch in tqdm(dev_dataloader):
    true_labels += (batch['labels'].numpy())[:, 0].flatten().tolist()
    batch = {k:v.type(torch.long).cuda() for k,v in batch.items()}

    with torch.no_grad():        
        outputs = model(**batch)

        loss, logits = outputs[:2]
        
        logits = logits.detach().cpu().numpy()
        total_loss += loss.item()

        predict_content = (logits.argmax(axis=-1))[:, 0].flatten().tolist()
        prediction_labels += predict_content

  # Calculate the average loss over the training data.
  dev_avg_epoch_loss = total_loss / len(dev_dataloader)
  #print((true_labels))
  #print((prediction_labels))
  #print(len(test_dataloader))
  dev_acc = accuracy_score(true_labels, prediction_labels)

  print("Epoch: %d  dev_accuracy: %.5f"%(epoch_i, dev_acc))

In [ ]:
dev= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/국립국어원/SKT_BoolQ_Dev.tsv", delimiter= '\t')
dev['prediction']=prediction_labels

dev['prediction'] = dev['prediction'].apply(lambda x: 1 if x==3966 else x)
dev['prediction'] = dev['prediction'].apply(lambda x: 0 if x==12958 else x)

dev[50:60]

,ID,Text,Question,"Answer(FALSE = 0, TRUE = 1)",prediction
50,51,경단(瓊團)은 구슬 모양 떡이다. 찹쌀가루나 찰수수 가루 등을 반죽해 밤톨만 한 크...,경단은 떡의 한 종류인가요?,1,1
51,52,1923년은 초창기 한국영화의 개척자로서 높이 평가되는 안종화가 등장하는 해이기도 ...,안종화의 직업은 변사(辯士)였나요?,0,1
52,53,"숨을 들이쉴 때는 복부가 부풀어 앞부분의 기문이 열리고 뒷부분의 기문은 닫히며, 이...",앞부분의 기문과 뒷부분의 기문은 동시에 열리지 않는다.,1,0
53,54,"구례 화엄사 각황전 앞 석등은 전라남도 구례군 화엄사 각황전 앞에 있는, 남북국 시...",구례 화엄사 각황전 앞 석등은 신라 시대에 지어졌나요?,1,1
54,55,뮬(Mule)은 뒤가 없고 앞은 마감되어 있는 신발 스타일을 지칭하는 프랑스어이다....,뮬의 어원은 프랑스어에서 유래했나요?,0,1
55,56,"2017년 6월 2일, 정규 데뷔 앨범 Dua Lipa 를 발매했다. 데뷔 앨범 D...",두아리파는 정규 데뷔 앨범 'Dua Lipa'의 뮤직비디오로 전 세계에 이름을 알리었다.,0,0
56,57,"종현(본명: 김종현, 1990년 4월 8일 ~ 2017년 12월 18일)은 대한민국...",종현은 솔로 활동을 한 적이 있는가?,1,1
57,58,"이데아론은 플라톤이 처음 주장한 형이상학 이론이다. 플라톤에 따르면, 이데아론에서 ...",이데아는 현상 세계를 의미하는가?,0,0
58,59,"여성의 리얼 스키니 진, 스키니팬츠가 계속 유행하고 있으며 아름다움을 돋보이게 할 ...",2010년대 이후 스키니진은 출시가 중단되었다.,0,0
59,60,사라예보 사건은 1914년 6월 28일 현재의 보스니아 헤르체고비나의 수도인 사라예...,제1차 세계대전 이후에 사라예보 사건이 일어났나요?,0,0


In [ ]:
dev.to_excel('dev_엑셀.xlsx')

In [ ]:
#######################korquadboolq 데이터셋

batch_size = 8

collator = T5_Classification_Collator(use_tokenizer=tokenizer, max_sequence_len=512)

print("Load Dataset...")
train_dataset = SKT_BoolQ('/content/drive/MyDrive/Colab Notebooks/국립국어원/KorQuAD+SKT.tsv')
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collator)

dev_dataset = SKT_BoolQ('/content/drive/MyDrive/Colab Notebooks/국립국어원/SKT_BoolQ_Dev.tsv')
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, collate_fn=collator)

epochs = 10
total_steps = len(train_dataloader) * epochs

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )
#scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                            num_warmup_steps = len(train_dataloader), # Default value in run_glue.py
#                                            num_training_steps = total_steps)
scheduler = get_constant_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = len(train_dataloader))

print("Training...")

for epoch_i in range(epochs):
  print("-------------------------------------------------------------")
  # train
  model.train()

  true_labels = []
  prediction_labels = []
  total_loss = 0

  for batch in tqdm(train_dataloader):
    true_labels += batch['labels'].numpy().flatten().tolist()
    batch = {k:v.type(torch.long).cuda() for k,v in batch.items()}

    model.zero_grad()
    outputs = model(**batch)

    loss, logits = outputs[:2]
    total_loss += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()

    logits = logits.detach().cpu().numpy()

    prediction_labels += logits.argmax(axis=-1).flatten().tolist()

  train_avg_epoch_loss = total_loss / len(train_dataloader)
  train_acc = accuracy_score(true_labels, prediction_labels)

  prediction_labels = []
  true_labels = []
  total_loss = 0

  model.eval()

  # Evaluate data for one epoch
  for batch in tqdm(dev_dataloader):
    true_labels += (batch['labels'].numpy())[:, 0].flatten().tolist()
    batch = {k:v.type(torch.long).cuda() for k,v in batch.items()}

    with torch.no_grad():        
        outputs = model(**batch)

        loss, logits = outputs[:2]
        
        logits = logits.detach().cpu().numpy()
        total_loss += loss.item()

        predict_content = (logits.argmax(axis=-1))[:, 0].flatten().tolist()
        prediction_labels += predict_content

  # Calculate the average loss over the training data.
  dev_avg_epoch_loss = total_loss / len(dev_dataloader)
  #print(len(true_labels))
  #print(len(prediction_labels))
  #print(len(dev_dataloader))
  dev_acc = accuracy_score(true_labels, prediction_labels)

  print("Epoch: %d  train_loss: %.5f  train_acc: %.5f  dev_loss: %.5f  dev_acc: %.5f"%(epoch_i, train_avg_epoch_loss, train_acc, dev_avg_epoch_loss, dev_acc))



Load Dataset...
Incorrectly formatted.
Incorrectly formatted.
Incorrectly formatted.
Incorrectly formatted.
Incorrectly formatted.
Incorrectly formatted.
Incorrectly formatted.
Incorrectly formatted.
Incorrectly formatted.
Incorrectly formatted.
Incorrectly formatted.
Incorrectly formatted.
Training...
-------------------------------------------------------------


  0%|          | 0/1082 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
/content/dataset.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs.update({'labels':torch.tensor(labels)})
100%|██████████| 88/88 [00:06<00:00, 14.02it/s]


Epoch: 0  train_loss: 68.75174  train_acc: 0.08381  dev_loss: 16.81616  dev_acc: 0.00000
-------------------------------------------------------------


100%|██████████| 88/88 [00:06<00:00, 14.06it/s]


Epoch: 1  train_loss: 8.32456  train_acc: 0.49526  dev_loss: 5.04383  dev_acc: 0.00000
-------------------------------------------------------------


...

Epoch: 7  train_loss: 0.38626  train_acc: 0.74934  dev_loss: 0.36040  dev_acc: 0.46571
-------------------------------------------------------------


100%|██████████| 88/88 [00:06<00:00, 14.15it/s]


Epoch: 8  train_loss: 0.37851  train_acc: 0.75090  dev_loss: 0.38280  dev_acc: 0.46571
-------------------------------------------------------------


100%|██████████| 88/88 [00:06<00:00, 14.11it/s]

Epoch: 9  train_loss: 0.37101  train_acc: 0.76095  dev_loss: 0.35232  dev_acc: 0.48571


In [ ]:
#############korquad+SKT 하나의 데이터로 합치기

import pandas as pd

train1= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/국립국어원/SKT_BoolQ_Train.tsv", delimiter= '\t')
train2= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/국립국어원/KorQuADBoolQ_v1.0.tsv", delimiter= '\t')

train3= pd.concat([train1, train2])
train3.to_csv('sample_file.txt', index=False, header=None, sep="\t")
train3.to_excel('sample_file.txt')

,ID,Text,Question,"Answer(FALSE = 0, TRUE = 1)"
0,1,"이 작품은 라단조, Sehr gehalten(아주 신중하게), 4/4박자의 부드러운...","이 곡의 주요 주제는 고뇌와 갈망 동기, 청춘의 사랑 동기인가?",1
1,2,서일본 여객철도 어번 네트워크의 한 축을 구성하고 있다. 오사카 시가지 남부의 철도...,간사이 국제공항이 개통된 해는 1994년이다.,0
2,3,대낮 시간대에는 덴노지 역 ~ 히네노 역 간에서 쾌속과 보통이 각각 시간당 6편씩 ...,간사이 본선 JR 난바역 발착 열차가 존재했던 마지막 해는 2008년이다.,1
3,4,덴노지 역 구내에서는 야마토지 선과 평면 교차하여 단선으로 운전하는 한와 선과의 단...,복선화가 시간표 혼란을 막은 선은 2008년이다.,0
4,5,기세이 본선 (기노쿠니 선)에 직통하는 특급 구로시오·슈퍼 구로시오·오션애로우 호와...,기본적으로 6량 편성이지만 승객이 많을 때 운행되는 량 수는 임시 열차인가?,0
5,6,1984년 9월부터 덴노지 역 ~ 히네노 역간에 운전을 개시한 '홈 라이너 이즈미'...,한와 라이너가 휴일 시간대 운전이 중단된 해는 2009년인가?,1
6,7,직통 쾌속은 2008년 3월 15일의 시간표 개정으로 평일 아침 출근 시간대에 신설...,직통 쾌속이 사용하는 차량은 직통 쾌속인가?,0
7,8,운전 구간 내의 모든 역에 정차한다. 덴노지 역 및 히네노 역 등에시 시발하여 오토...,"쓰루가오카 역, 스기모토초 역, 구마토리 역, 히네노 역 등에 있는 것은 대피 설비이다.",1
8,9,"헨리 구스타프 몰래슨(영어: Henry Gustav Molaison, 1926년 2...",미국의 헨리 구스타프 몰래슨 환자의 뇌전증을 치료하기 위해 환자의 뇌에서 제거된 부...,1
9,10,"그의 기억상실증상에도 불구하고, 몰래슨은 지능 검사에서는 지극히 정상이었는데, 이것...",몰래슨이 기억상실증을 앓고 있었음에도 불구하고 일반 사람들과 크게 다르지 않고 정상...,0


In [ ]:
!ls -al

total 28
drwxr-xr-x 1 root root 4096 Sep 30 02:12 .
drwxr-xr-x 1 root root 4096 Sep 30 02:06 ..
drwxr-xr-x 4 root root 4096 Sep 16 13:39 .config
-rw-r--r-- 1 root root 1568 Sep 30 02:12 dataset.py
drwx------ 6 root root 4096 Sep 30 02:10 drive
drwxr-xr-x 2 root root 4096 Sep 30 02:12 __pycache__
drwxr-xr-x 1 root root 4096 Sep 16 13:40 sample_data


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/국립국어원/'

/content/drive/MyDrive/Colab Notebooks/국립국어원


In [3]:
!python BoolQ_t5_base_test.py

Load T5 model...
Downloading: 100% 1.40M/1.40M [00:00<00:00, 8.72MB/s]
Downloading: 100% 1.74k/1.74k [00:00<00:00, 2.10MB/s]
Downloading: 100% 1.92k/1.92k [00:00<00:00, 2.58MB/s]
Downloading: 100% 599/599 [00:00<00:00, 837kB/s]
Downloading: 100% 944M/944M [00:21<00:00, 46.1MB/s]
Load Dataset...
Training...
-------------------------------------------------------------
  0% 0/459 [00:00<?, ?it/s]BoolQ_t5_base_test.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs.update({'labels':torch.tensor(labels)})
100% 459/459 [01:59<00:00,  3.83it/s]
100% 88/88 [00:06<00:00, 13.59it/s]
Epoch: 0  train_loss: 78.74239  train_acc: 0.00095  dev_loss: 31.49577  dev_acc: 0.00000
-------------------------------------------------------------
100% 459/459 [01:59<00:00,  3.85it/s]
100% 88/88 [00:06<00:00, 13.57it/s]
Epoch: 1  train_loss: 16.28850  t

In [ ]:
###########개체명 인식


In [ ]:
import pandas as pd
train_dataset = pd.read_csv('SKT_NER_DROP_Train.tsv', delimiter= '\t')
train_dataset

,ID,Text,Question,"Answer(FALSE = 0, TRUE = 1)",Question NER,Text NER
0,1,['로마 시대의 오리엔트의 범위는 제국 내에 동부 지방은 물론 제국 외부에 있는 다...,"오리엔트는 인도와 중국, 일본을 이루는 광범위한 지역을 지칭하는 단어로 쓰인다.",1,일본 오리엔트 중국 인도,동방 서유럽 옥시덴트 로마 제국 로마 시대 occident 일본 서방 오리엔트 중국...
1,2,['비글을 키우려면 비글이 뛰어놀수 있는 넓은 놀이공간 등을 확보하고 있는 단독주택...,비글은 넓고 뚤린 공간에서 키워야 한다.,1,비글,개 비글 다세대
2,3,['타이완 요리의 특징은 토속 요리(일본 통치 전)에서 기름을 많이 사용하는 다른 ...,타이완 요리는 다른 지역의 중국 요리처럼 기름을 많이 사용하는 것이다.,0,타이완 중국,타이완 기름 소금 일본 중국
3,4,['연하곤란은 음식물이 구강에서 식도로 넘어가는 과정에 문제가 생겨 음식을 원활히 ...,연하곤란이 생기면 식도가 막히나요?,0,연하곤란 식도,폐렴 구강 근육 식도 음식물 연하곤란 뇌 종양 식도협착
4,5,"['인문과학 또는 인문학(人文學, 영어: humanities)은 인간과 인간의 근원...",인문과학은 경험적인 접근을 주로 사용하는가?,0,인문과학,인문과학 영어 인문학 자연과학 人文學
...,...,...,...,...,...,...
3660,3661,"['사법시험은 법조인이 될 자격을 검정하는 시험이다.', ' 제1차(객관식), 제2...",사법 시험 합격 이후에는 사법연수원을 들어가야 하는가?,1,사법연수원,판사 변호사 세 번 제2차 법조인 제3차 사법연수원 제1차 검사
3661,3662,['한국사능력검정시험(韓國史能力檢定試驗)은 한국사에 대한 관심을 확산 · 심화시키는...,한국사능력검정시험(韓國史能力檢定試驗)은 국사편찬위원회가 주관하는가?,1,국사편찬위원회 韓國史能力檢定試驗 한국사능력검정시험,1급 3급 2006년 11월 25일부터 국사편찬위원회 홈페이지 韓國史能力檢定試驗 5...
3662,3663,"['한국교통안전공단(韓國交通安全公團, Korea Transportation Safe...",한국교통안전공단은 준정부기관인가?,1,한국교통안전공단,1979년 12월 28일 韓國交通安全公團 교통안전진흥공단법 교통안전진흥공단 TS 1...
3663,3664,"['변호사시험은 사법시험의 후신으로서, 조선변호사시험-고등고시 사법과-사법시험-변호...",변호사 시험은 2017년에 완전히 폐지되었는가?,1,2017년 변호사,조선 2017년 변호사 법학전문대학원 몇년간 변호사시험-고등고시 사법과
